# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119969e+02     1.562065e+00
 * time: 0.547011137008667
     1     1.123649e+01     9.759906e-01
 * time: 1.3527700901031494
     2    -1.216078e+01     9.933747e-01
 * time: 1.4359190464019775
     3    -3.409201e+01     7.201326e-01
 * time: 1.5619230270385742
     4    -4.780062e+01     5.489582e-01
 * time: 1.704921007156372
     5    -5.702223e+01     2.297897e-01
 * time: 1.8096129894256592
     6    -5.986452e+01     1.312746e-01
 * time: 1.8967230319976807
     7    -6.096868e+01     5.663038e-02
 * time: 1.9772770404815674
     8    -6.140162e+01     6.075280e-02
 * time: 2.0779950618743896
     9    -6.165150e+01     5.525736e-02
 * time: 2.160137176513672
    10    -6.184838e+01     2.926663e-02
 * time: 2.2485721111297607
    11    -6.199483e+01     2.190294e-02
 * time: 2.3298850059509277
    12    -6.206659e+01     1.413353e-02
 * time: 2.4227330684661865
    13    -6.212193e+01     1.245368e-02
 * time: 2.5015540122985

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671073
    AtomicLocal         -18.8557688
    AtomicNonlocal      14.8522646
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336823

    total               -62.261666459967